# Q-Learning 이론과 워크플로우

## 학습 목표
- 벨만 방정식과 Q-러닝의 핵심 개념을 구현 중심으로 체득
- Epsilon-Greedy 탐험전략을 설계·조정
- Q-Table 학습의 전체 워크플로우 이해

## 평가 기준
- Q-learning 업데이트 수식의 이해와 구현
- 탐험-활용 균형의 중요성 파악
- 하이퍼파라미터가 학습에 미치는 영향 분석

## A. 가치 기반 학습 요약

### 1. 강화학습의 기본 구성요소

강화학습은 **에이전트(Agent)**가 **환경(Environment)**과 상호작용하며 **보상(Reward)**을 최대화하는 **정책(Policy)**을 학습하는 과정입니다.

- **상태 (State, S)**: 환경의 현재 상황
- **행동 (Action, A)**: 에이전트가 취할 수 있는 선택
- **보상 (Reward, R)**: 행동에 대한 즉각적인 피드백
- **정책 (Policy, π)**: 상태에서 행동을 선택하는 전략
- **가치 함수 (Value Function)**: 미래 보상의 기댓값

### 2. 벨만 방정식 (Bellman Equation)

벨만 방정식은 현재 상태의 가치를 재귀적으로 정의합니다:

**상태 가치 함수:**
$$V^\pi(s) = \mathbb{E}[R_{t+1} + \gamma V^\pi(S_{t+1}) | S_t = s]$$

**행동 가치 함수 (Q-function):**
$$Q^\pi(s,a) = \mathbb{E}[R_{t+1} + \gamma Q^\pi(S_{t+1}, A_{t+1}) | S_t = s, A_t = a]$$

### 3. 최적 정책과 Q*

**최적 Q-함수:**
$$Q^*(s,a) = \max_\pi Q^\pi(s,a)$$

**최적 정책:**
$$\pi^*(s) = \arg\max_a Q^*(s,a)$$

즉, 최적 Q-함수를 알면 각 상태에서 가장 높은 Q-값을 갖는 행동을 선택하여 최적 정책을 얻을 수 있습니다.

## B. Q-Learning 알고리즘

### 1. Q-Learning 업데이트 수식

Q-Learning은 다음 업데이트 규칙을 사용하여 Q-함수를 학습합니다:

$$Q(s,a) \leftarrow Q(s,a) + \alpha \left[ r + \gamma \max_{a'} Q(s',a') - Q(s,a) \right]$$

여기서:
- $\alpha$: 학습률 (learning rate, 0 < α ≤ 1)
- $\gamma$: 할인 인수 (discount factor, 0 ≤ γ < 1)
- $r$: 즉각적인 보상
- $s'$: 다음 상태

### 2. 수식의 구성요소 분석

- **$Q(s,a)$**: 현재 Q-값 (현재 추정치)
- **$r + \gamma \max_{a'} Q(s',a')$**: TD 목표값 (Temporal Difference Target)
- **$r + \gamma \max_{a'} Q(s',a') - Q(s,a)$**: TD 오차 (Temporal Difference Error)
- **$\alpha$**: 새로운 정보를 얼마나 반영할지 결정

### 3. Off-Policy 학습

Q-Learning은 **off-policy** 알고리즘입니다:
- 행동 선택은 ε-greedy 정책 사용 (탐험)
- Q-값 업데이트는 greedy 정책 사용 (max 연산)
- 이로 인해 탐험하면서도 최적 정책을 학습 가능

## C. 에이전트 구성요소 - FrozenLake 예시

### 1. FrozenLake 환경 소개

FrozenLake는 4x4 (또는 8x8) 격자 세계에서 구멍을 피해 목표 지점에 도달하는 게임입니다.

```
SFFF    S: 시작점 (Start)
FHFH    F: 얼어붙은 표면 (Frozen)
FFFH    H: 구멍 (Hole)
HFFG    G: 목표 (Goal)
```

### 2. 상태 공간 (State Space)
- **크기**: 4x4 = 16개 상태 (8x8의 경우 64개)
- **표현**: 정수 0~15 (격자 위치를 일렬로 번호화)
- **예시**: 
  - 상태 0: (0,0) 시작점
  - 상태 15: (3,3) 목표점

### 3. 행동 공간 (Action Space)
- **크기**: 4개 행동
- **행동**:
  - 0: 왼쪽 (Left)
  - 1: 아래 (Down)
  - 2: 오른쪽 (Right)
  - 3: 위 (Up)

### 4. 보상 구조 (Reward Structure)
- **목표 도달**: +1
- **구멍에 빠짐**: 0
- **일반 이동**: 0
- **에피소드 종료**: 목표 도달 또는 구멍에 빠질 때

### 5. 전이 확률 (Transition Probability)
- **is_slippery=True**: 의도한 방향으로 1/3, 수직 방향으로 각각 1/3씩
- **is_slippery=False**: 의도한 방향으로 100% 이동

### 6. Q-Table 구조
- **모양**: (16, 4) - 16개 상태 × 4개 행동
- **초기화**: 모든 값을 0으로 설정
- **업데이트**: 경험한 (s, a) 쌍에 대해서만 업데이트

## D. 탐험 전략 (Exploration Strategy)

### 1. 탐험-활용 딜레마 (Exploration-Exploitation Dilemma)

강화학습에서 가장 중요한 문제 중 하나:
- **활용 (Exploitation)**: 현재 알고 있는 최선의 행동 선택
- **탐험 (Exploration)**: 새로운 행동을 시도하여 더 나은 정책 발견

### 2. ε-Greedy 정책

가장 단순하고 효과적인 탐험 전략:

```python
if random() < epsilon:
    action = random_action()  # 탐험
else:
    action = argmax(Q[state])  # 활용
```

### 3. ε 스케줄링 전략

학습 초기에는 많이 탐험하고, 후기에는 활용에 집중:

**선형 감쇠:**
```python
epsilon = max(epsilon_min, epsilon_start - (episode / max_episodes) * (epsilon_start - epsilon_min))
```

**지수 감쇠:**
```python
epsilon = max(epsilon_min, epsilon * epsilon_decay)
```

### 4. Tie-Breaking (동점 처리)

여러 행동이 같은 최대 Q-값을 가질 때 무작위로 선택:

```python
q_values = Q[state]
max_q = np.max(q_values)
best_actions = np.where(q_values == max_q)[0]
action = np.random.choice(best_actions)
```

### 5. 탐험 전략의 중요성

- **충분한 탐험**: 모든 상태-행동 쌍을 경험
- **점진적 감소**: 학습이 진행될수록 활용 비중 증가
- **최소값 보장**: 완전히 탐험을 멈추지 않음

## E. Q-Learning 워크플로우

### 1. 초기화 단계
```python
# 1. Q-Table 초기화 (보통 0으로)
Q = np.zeros((state_size, action_size))

# 2. 하이퍼파라미터 설정
alpha = 0.1          # 학습률
gamma = 0.95         # 할인 인수
epsilon = 1.0        # 초기 탐험률
epsilon_min = 0.01   # 최소 탐험률
epsilon_decay = 0.995 # 탐험률 감쇠

# 3. 학습 설정
num_episodes = 5000
max_steps = 100
```

### 2. 에피소드 루프
```python
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0
    
    for step in range(max_steps):
        # 행동 선택 (ε-greedy)
        action = choose_action(state, epsilon)
        
        # 환경에서 행동 실행
        next_state, reward, done, info = env.step(action)
        
        # Q-Table 업데이트
        update_q_table(state, action, reward, next_state, done)
        
        # 상태 전이
        state = next_state
        total_reward += reward
        
        if done:
            break
    
    # ε 감쇠
    epsilon = max(epsilon_min, epsilon * epsilon_decay)
```

### 3. Q-Table 업데이트 함수
```python
def update_q_table(state, action, reward, next_state, done):
    current_q = Q[state, action]
    
    if done:
        target_q = reward
    else:
        target_q = reward + gamma * np.max(Q[next_state])
    
    Q[state, action] = current_q + alpha * (target_q - current_q)
```

### 4. 로깅과 모니터링
```python
# 에피소드별 기록
episode_rewards = []
success_rates = []

# 이동 평균 계산 (예: 100 에피소드 윈도우)
window_size = 100
if len(episode_rewards) >= window_size:
    moving_avg = np.mean(episode_rewards[-window_size:])
    success_rate = np.mean(success_episodes[-window_size:])
```

## F. 하이퍼파라미터 감각

### 1. 학습률 (Learning Rate, α)

**역할**: 새로운 정보를 얼마나 빠르게 반영할지 결정

- **α가 클 때 (0.5~1.0)**:
  - 빠른 학습, 하지만 불안정
  - 최근 경험에 크게 의존
  - 노이즈에 민감

- **α가 작을 때 (0.01~0.1)**:
  - 안정적 학습, 하지만 느림
  - 과거 경험을 오래 기억
  - 변화하는 환경에 적응 어려움

**권장값**: 0.1~0.3 (환경에 따라 조정)

### 2. 할인 인수 (Discount Factor, γ)

**역할**: 미래 보상의 중요도 결정

- **γ가 클 때 (0.95~0.99)**:
  - 장기적 관점, 미래 보상 중시
  - 복잡한 전략 학습 가능
  - 수렴이 느릴 수 있음

- **γ가 작을 때 (0.8~0.9)**:
  - 단기적 관점, 즉각적 보상 중시
  - 빠른 수렴
  - 최적이 아닐 수 있음

**권장값**: 0.95~0.99 (에피소드 길이에 따라)

### 3. 탐험률 스케줄

**ε_start (초기값)**:
- 1.0: 완전한 무작위 탐험으로 시작
- 0.5~0.8: 부분적 사전 지식 활용

**ε_min (최소값)**:
- 0.01~0.05: 항상 약간의 탐험 유지
- 0.1: 더 많은 탐험 (동적 환경)

**ε_decay (감쇠율)**:
- 0.995: 천천히 감소
- 0.99: 보통 속도
- 0.98: 빠르게 감소

### 4. 하이퍼파라미터 조합 예시

**보수적 설정 (안정적, 느림)**:
```python
alpha = 0.1
gamma = 0.99
epsilon_decay = 0.998
```

**적극적 설정 (빠름, 불안정할 수 있음)**:
```python
alpha = 0.3
gamma = 0.95
epsilon_decay = 0.99
```

**균형 설정 (일반적 추천)**:
```python
alpha = 0.1
gamma = 0.95
epsilon_decay = 0.995
```

## G. 검정 지표 (Evaluation Metrics)

### 1. 에피소드 리턴 (Episode Return)

**정의**: 한 에피소드에서 얻은 총 보상
```python
episode_return = sum(rewards_in_episode)
```

**해석**:
- FrozenLake에서는 0 (실패) 또는 1 (성공)
- 이동 평균으로 학습 진행 상황 파악

### 2. 성공률 (Success Rate)

**정의**: 목표에 도달한 에피소드의 비율
```python
success_rate = (successful_episodes / total_episodes) * 100
```

**윈도우 기반 성공률**:
```python
window_success_rate = np.mean(last_100_episodes) * 100
```

### 3. 이동 평균 (Moving Average)

**목적**: 노이즈 제거, 트렌드 파악
```python
def moving_average(data, window_size):
    return [np.mean(data[i:i+window_size]) 
            for i in range(len(data)-window_size+1)]
```

### 4. 학습 곡선 시각화

**에피소드 리턴 곡선**:
- X축: 에피소드 번호
- Y축: 이동 평균 리턴
- 상승 트렌드 → 학습 진행

**성공률 곡선**:
- X축: 에피소드 번호
- Y축: 윈도우 성공률 (%)
- 목표: 80% 이상 달성

### 5. Q-Table 분석 (선택사항)

**Q-값 히트맵**:
```python
max_q_values = np.max(Q, axis=1).reshape(4, 4)
plt.imshow(max_q_values, cmap='viridis')
```

**정책 시각화**:
```python
policy = np.argmax(Q, axis=1)
action_symbols = ['←', '↓', '→', '↑']
```

### 6. 성능 벤치마크

**FrozenLake 4x4 (is_slippery=True) 기준**:
- 무작위 정책: ~5% 성공률
- 학습된 정책: 70~80% 성공률 (목표)
- 최적 정책: 이론적으로 ~85% (환경의 확률적 특성 때문)

**수렴 기준**:
- 100 에피소드 윈도우에서 성공률이 일정 수준 유지
- 이동 평균 리턴이 더 이상 증가하지 않음
- ε가 최소값에 도달

## 요약 및 다음 단계

### 핵심 개념 정리

1. **Q-Learning은 model-free, off-policy 알고리즘**
2. **벨만 방정식 기반의 점진적 가치 함수 학습**
3. **ε-greedy로 탐험-활용 균형 조절**
4. **하이퍼파라미터 튜닝이 성능에 큰 영향**

### 실습 준비사항

다음 노트북에서는:
- 실제 FrozenLake 환경에서 Q-Learning 구현
- 다양한 하이퍼파라미터 실험
- 학습 곡선 시각화 및 분석
- 학습된 정책의 평가 및 시연

### 성공 기준

- [ ] Q-Learning 업데이트 수식 이해
- [ ] ε-greedy 정책의 동작 원리 파악
- [ ] 하이퍼파라미터가 학습에 미치는 영향 이해
- [ ] 평가 지표의 의미와 해석 방법 숙지

이제 `02_Qlearning_FrozenLake_Training.ipynb`로 넘어가서 실제 구현을 시작해보겠습니다!